In [13]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import time
# import custom libraries
import sys
sys.path.append("C:\\DATA\\Tasks\\lib\\hk")
import hk_psql
import dash_bootstrap_components

import plotly.express as px  # (version 4.7.0 or higher)
import plotly.graph_objects as go

ADD_DATA = "C:\\DATA\\data\\raw\\mimic4\\lookup\\"

%load_ext autoreload
%autoreload 2

conn = hk_psql.connect_psql('mimic')


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
DATABASE CONNECTED !!!


In [22]:
sel_icustay = [{'id_careunit': 'Medical Intensive Care Unit (MICU)', 'id_los_min': '24', 'id_los_max': '240'},
 {'id_careunit': 'Trauma SICU (TSICU)', 'id_los_min': '24', 'id_los_max': '240'},
 {'id_careunit': 'Surgical Intensive Care Unit (SICU)', 'id_los_min': '24', 'id_los_max': '240'}]

sel_icustay = [
    # {'id_careunit': 'ICU', 'id_los_min': '24', 'id_los_max': '1000'},
    {'id_careunit': 'Cardiac Surgery', 'id_los_min': '0', 'id_los_max': '1000'},
]


filters_core = {
        'age':[20,91],
        'gender':['M','F'],
        'icustays':pd.DataFrame.from_dict(sel_icustay),
        'adm_types':[],
        'adm_locs':[],
        'dis_locs':[],

    }
# print(filters)
q_core = hk_psql.bq_core(filters_core)
# print(q_core)
df_filtered_core = hk_psql.query(q_core, conn)
print('Hojjatak')
hadm_ids_core = df_filtered_core['hadm_id'].unique().tolist()
print(df_filtered_core)
print('len after core: ',len(hadm_ids_core))

       id_careunit id_los_min id_los_max
0  Cardiac Surgery          0       1000
hi
hi2
       id_careunit id_los_min id_los_max
0  Cardiac Surgery          0       1000
hi3
and(

            (
                 strpos(tra.careunit,'Cardiac Surgery')>0
                and
                (tra.outtime between (tra.intime+ interval '1' HOUR * 0 )
                 AND (tra.intime+ interval '1' HOUR * 1000 )             )
            )
            )
done
--- 0.6225137710571289 seconds ---
Hojjatak
       subject_id   hadm_id   stay_id  \
0        14990224  20000147  30503572   
1        14990224  20000147  30503572   
2        14990224  20000147  30503572   
3        17038060  20005241  30969401   
4        12377638  20006999  32160575   
...           ...       ...       ...   
14053    17660536  29997844  39862305   
14054    17660536  29997844  34886849   
14055    17660536  29997844  39862305   
14056    17624950  29998113  32711700   
14057    19989305  29998115  36976872   

        

In [23]:
# **************** STEP 2: filter hosp codes (drg , icd)
    
filters_codes = {'drgcodes': pd.DataFrame(),
                    'diagnoses_icd': pd.DataFrame.from_dict({'icd_code': {770: 'R6520  ',
                                                        1058: 'A4151  ',
                                                        3330: 'A4152  ',
                                                        327: '99591  ',
                                                        170: '99592  ',
                                                        2123: 'A4159  ',
                                                        3216: 'A408   ',
                                                        305: 'R6521  ',
                                                        2077: 'A4181  ',
                                                        3539: 'A411   ',
                                                        2838: 'A4102  ',
                                                        1816: 'A4101  ',
                                                        2299: '77181  ',
                                                        189: 'A419   ',
                                                        1406: 'A4189  '}, 'icd_version': {770: 10, 1058: 10, 3330: 10, 327: 9, 170: 9, 2123: 10, 3216: 10, 305: 10, 2077: 10, 3539: 10, 2838: 10, 1816: 10, 2299: 9, 189: 10, 1406: 10}, 'long_title': {770: 'Severe sepsis without septic shock', 1058: 'Sepsis due to Escherichia coli [E. coli]', 3330: 'Sepsis due to Pseudomonas', 327: 'Sepsis', 170: 'Severe sepsis', 2123: 'Other Gram-negative sepsis', 3216: 'Other streptococcal sepsis', 305: 'Severe sepsis with septic shock', 2077: 'Sepsis due to Enterococcus', 3539: 'Sepsis due to other specified staphylococcus', 2838: 'Sepsis due to Methicillin resistant Staphylococcus aureus', 1816: 'Sepsis due to Methicillin susceptible Staphylococcus aureus', 2299: 'Septicemia [sepsis] of newborn', 189: 'Sepsis, unspecified organism', 1406: 'Other specified sepsis'}, 'count': {770: 1226, 1058: 849, 3330: 146, 327: 2988, 170: 5291, 2123: 315, 3216: 155, 305: 3175, 2077: 329, 3539: 133, 2838: 192, 1816: 403, 2299: 277, 189: 4766, 1406: 563}}),
                                                        
                        'procedures_icd': pd.DataFrame()}

filters_codes = {
    'drgcodes': pd.DataFrame(), 
    'diagnoses_icd': pd.read_csv(ADD_DATA+'saved_diag_MI.csv'),
    'procedures_icd': pd.DataFrame(), 

    # 'procedures_icd': pd.read_csv(ADD_DATA+'saved_proc_cabg.csv'),
}

q = hk_psql.bq_codes(filters_codes, hadm_ids_core)
print('sdfds')
if len(filters_codes['procedures_icd'])+len(filters_codes['diagnoses_icd'])+len(filters_codes['drgcodes'])==0:
    df_filtered_codes = df_filtered_core
else:
    df_filtered_codes = hk_psql.query(q, conn)
# print(df_filtered_codes)
# df_filtered_core.to_csv(ADD_DATA+'temp.csv')
hadm_ids_core_code = df_filtered_codes['hadm_id'].unique().tolist()
print('len after code: ', len(hadm_ids_core_code))

# q=f"""
# SELECT * FROM mimic_icu.icustays


# where los>1 and los<10 
# """
# df_filtered_codes = hk_psql.query(q, conn)
# print(df_filtered_codes)
# df_filtered_core = pd.merge(df_filtered_core, df_filtered_codes[['hadm_id','stay_id','los']], how='inner',on='hadm_id')
# print(df_filtered_core)

# hadm_ids_core_code = df_filtered_core['hadm_id'].unique().tolist()
# print()
# print('len after code: ', len(hadm_ids_core_code))


Unnamed: 0      int64
icd_code        int64
icd_version     int64
long_title     object
count           int64
dtype: object
Unnamed: 0      int64
icd_code       object
icd_version     int64
long_title     object
count           int64
dtype: object
QUERY CODES 11 >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
HOOJJAT
['021209W', '3613', '3611', '3612', '3615', '3614']

    SELECT 
        drg.hadm_id,
        drg.drg_code,
        diag.icd_code as diag_icd,
        diag.seq_num as diag_seq,
        proc.icd_code as proc_icd,
        proc.seq_num as proc_seq


    FROM mimic_hosp.drgcodes drg

            inner join mimic_hosp.diagnoses_icd diag
            on diag.hadm_id=drg.hadm_id

            inner join mimic_hosp.procedures_icd proc
            on proc.hadm_id=drg.hadm_id

            where
                TRUE

            and ( drg.hadm_id in (20000147, 20005241, 20006999, 20008851, 20010499, 20012858, 20013602, 20014283, 20017597, 20022292, 20022786, 20023678, 20025569, 20027082, 20029054,

In [24]:
len(df_filtered_codes)

2492

In [4]:
# **************** STEP 3: Hosp filters -> hadm_ids
lab_table = [
    {'id_item': 50813, 'id_label': 'Lactate', 'id_fluid': 'Blood', 'id_cat': 'Blood Gas', 'id_range': '0.5-2.0', 'id_unit': 'mmol/L', 'id_condition': 'X>0', 'id_counts': 1}, 
# {'id_item': 50889, 'id_label': 'C-Reactive Protein', 'id_fluid': 'Blood', 'id_cat': 'Chemistry', 'id_range': '0.0-5.0', 'id_unit': 'mg/L', 'id_condition': 'X>0', 'id_counts': 10}, 
{'id_item': 51003, 'id_label': 'Troponin T', 'id_fluid': 'Blood', 'id_cat': 'Chemistry', 'id_range': '0.0-0.01', 'id_unit': 'ng/mL', 'id_condition': 'X>0', 'id_counts': 1}, 
# {'id_item': 50963, 'id_label': 'NTproBNP', 'id_fluid': 'Blood', 'id_cat': 'Chemistry', 'id_range': '0.0-177.0', 'id_unit': 'pg/mL', 'id_condition': 'X>0', 'id_counts': 1}, 
# {'id_item': 50924, 'id_label': 'Ferritin', 'id_fluid': 'Blood', 'id_cat': 'Chemistry', 'id_range': '13.0-150.0', 'id_unit': 'ng/mL', 'id_condition': 'X>0', 'id_counts': 1}, 
# {'id_item': 50909, 'id_label': 'Cortisol', 'id_fluid': 'Blood', 'id_cat': 'Chemistry', 'id_range': '2.0-20.0', 'id_unit': 'ug/dL', 'id_condition': 'X>0', 'id_counts': 1},
]       


filters_hosp = {
    'labevents': pd.DataFrame.from_dict(lab_table),

}
filters_hosp = {
    'labevents': pd.DataFrame(),

}
# print (lab_table)

q_hosp = hk_psql.bq_hosp(hadm_ids_core_code, filters_hosp)
print('hi fdgdgf')
# print(q_hosp)
df_filtered_hosp = hk_psql.query(q_hosp, conn)
print(df_filtered_hosp)
df_filtered_core = pd.merge(df_filtered_hosp[['hadm_id']], df_filtered_core[['subject_id','hadm_id','anchor_age','gender','careunit','icu_los','stay_id']], how='inner',on=['hadm_id'])
df_filtered = pd.merge(df_filtered_hosp, df_filtered_core[['subject_id','hadm_id','anchor_age','gender','careunit','icu_los','stay_id']], how='inner',on=['hadm_id'])

print(df_filtered_core)
# print(df_filtered_hosp)        
print(df_filtered)

hadm_ids = df_filtered_core['hadm_id'].unique().tolist()
stay_ids = df_filtered_core['stay_id'].unique().tolist()

sub_ids = df_filtered_core['subject_id'].unique().tolist()
print('len after hosp: ',len(hadm_ids))

hi fdgdgf
--- 12.421241521835327 seconds ---
         hadm_id  itemid  counts
0       20000147   50802       8
1       20000147   50804       8
2       20000147   50806       2
3       20000147   50808       6
4       20000147   50809       7
...          ...     ...     ...
225629  29998113   52073       1
225630  29998113   52074       1
225631  29998113   52075       1
225632  29998113   52135       1
225633  29998113   52172       6

[225634 rows x 3 columns]
         hadm_id  subject_id  anchor_age gender         careunit   icu_los  \
0       20000147    14990224          71      M  Cardiac Surgery  1.205498   
1       20000147    14990224          71      M  Cardiac Surgery  1.205498   
2       20000147    14990224          71      M  Cardiac Surgery  1.205498   
3       20000147    14990224          71      M  Cardiac Surgery  1.205498   
4       20000147    14990224          71      M  Cardiac Surgery  1.205498   
...          ...         ...         ...    ...              ...

In [5]:
cohort_name = 'CABG'
hk_psql.save_cohort(hadm_ids, cohort_name)

--- 0.0512995719909668 seconds ---
mimic_core transfers
--- 0.24004244804382324 seconds ---
mimic_core admissions
--- 0.17763447761535645 seconds ---
--- 0.04255485534667969 seconds ---
mimic_hosp labevents
--- 22.988525390625 seconds ---
mimic_hosp diagnoses_icd
--- 0.4586215019226074 seconds ---
mimic_hosp procedures_icd
--- 0.18216776847839355 seconds ---
mimic_hosp drgcodes
--- 0.2022991180419922 seconds ---
mimic_hosp emar
--- 4.784366130828857 seconds ---
--- 0.140977144241333 seconds ---
mimic_icu chartevents
--- 68.25221538543701 seconds ---
mimic_icu inputevents
--- 4.1250715255737305 seconds ---
mimic_icu procedureevents
--- 0.2819669246673584 seconds ---
mimic_icu icustays
--- 0.08296656608581543 seconds ---


In [6]:
# load a cohort
cohort_name = 'cabg'
schema_core = cohort_name+'_mimic_core'
df_overview = hk_psql.summary(schema_core, conn)
hadm_ids = df_overview['hadm_id'].unique()

q=f"""
    select * from mimic_core.patients t1
    inner join {schema_core}.admissions t2
    on t1.subject_id = t2.subject_id
    where t2.hadm_id in ({str(hadm_ids.tolist())[1:-1]})
    
    """
# print(q)

df_patients = hk_psql.query(q, conn)

q=f"""
        select * from {schema_core}.transfers         
        """
# print(q)

df_transfers = hk_psql.query(q, conn)

q=f"""
        select * from {cohort_name+'_mimic_icu'}.icustays         
        """
# print(q)

df_icustay = hk_psql.query(q, conn)

# LAB EVENTS
q=f"""
select * from {cohort_name+'_mimic_hosp'}.labevents         
"""

df_le = hk_psql.query(q, conn)

# CHART EVENTS
q=f"""
select * from {cohort_name+'_mimic_icu'}.chartevents         
"""

df_che = hk_psql.query(q, conn)


df_patients = df_patients.loc[:,~df_patients.columns.duplicated()]
print(df_patients['subject_id'])
print(df_transfers['subject_id'])


cols_to_use = [x for x in df_transfers.columns.tolist() if x not in df_patients.columns.tolist()]
cols_to_use.append('subject_id')
print(cols_to_use)
df_demo = pd.merge(df_patients, df_transfers[cols_to_use], how='inner',  on='subject_id')

print(df_demo)
df_le_main = df_le.copy()

--- 0.01703619956970215 seconds ---
--- 0.09789180755615234 seconds ---
--- 0.06304383277893066 seconds ---
--- 0.015955448150634766 seconds ---
--- 11.415144205093384 seconds ---
--- 83.46001815795898 seconds ---
0       10127185
1       10651616
2       13635761
3       15610840
4       15665720
          ...   
3363    19814368
3364    19894936
3365    19901410
3366    19950645
3367    19984270
Name: subject_id, Length: 3368, dtype: int64
0        19695954
1        12397875
2        16586569
3        16586569
4        14062718
           ...   
18941    18900837
18942    14877863
18943    10876550
18944    15585131
18945    15600650
Name: subject_id, Length: 18946, dtype: int64
['transfer_id', 'eventtype', 'careunit', 'intime', 'outtime', 'subject_id']
       subject_id gender  anchor_age  anchor_year anchor_year_group   dod  \
0        10127185      M          60         2141       2008 - 2010  None   
1        10127185      M          60         2141       2008 - 2010  None   
2  

In [7]:
df_che_main = df_che.copy()

In [12]:
df_che[df_che['itemid']==228445]


,subject_id,hadm_id,stay_id,charttime,storetime,itemid,value,valuenum,valueuom,warning
9860517,17773309,26323835,31134933,2113-03-23 20:20:00,2113-03-23 20:20:00,228445,Clean and Dry,NaN,None,0
9860533,17773309,26323835,31134933,2113-03-23 22:00:00,2113-03-23 22:06:00,228445,Clean and Dry,NaN,None,0
9860618,17773309,26323835,31134933,2113-03-24 00:00:00,2113-03-24 00:58:00,228445,Clean and Dry,NaN,None,0
9860715,17773309,26323835,31134933,2113-03-24 04:00:00,2113-03-24 04:15:00,228445,Clean and Dry,NaN,None,0
9860817,17773309,26323835,31134933,2113-03-24 08:00:00,2113-03-24 08:52:00,228445,Clean and Dry,NaN,None,0
9860869,17773309,26323835,31134933,2113-03-24 12:00:00,2113-03-24 14:16:00,228445,Clean and Dry,NaN,None,0
9860947,17773309,26323835,31134933,2113-03-24 16:00:00,2113-03-24 15:57:00,228445,Clean and Dry,NaN,None,0
9861041,17773309,26323835,31134933,2113-03-24 20:00:00,2113-03-24 19:55:00,228445,Clean and Dry,NaN,None,0
11001148,13480254,29235164,35997495,2172-02-29 13:32:00,2172-02-29 13:33:00,228445,Clean and Dry,NaN,None,0
11001282,13480254,29235164,35997495,2172-02-29 16:00:00,2172-02-29 18:01:00,228445,Clean and Dry,NaN,None,0


In [ ]:
# read desired itemids from csv files

ADD_DATA = "C:\\DATA\\data\\raw\\mimic4\\lookup\\"
df= pd.read_csv(ADD_DATA+'d_lookup_le.csv')
le_itemdids = df['itemid']


df= pd.read_csv(ADD_DATA+'d_lookup_che.csv')
che_itemdids = df['itemid']


grid='1H'


In [ ]:
df_le = df_le_main
df_che = df_che_main
print(len(df_demo['hadm_id'].unique()))
print(len(df_le['hadm_id'].unique()))
print(len(df_icustay['stay_id'].unique()))

icustay_ids = len(df_icustay['stay_id'].unique())

df_le = df_le[df_le['itemid'].isin(le_itemdids)]
df_che = df_che[df_che['itemid'].isin(che_itemdids)]

print(df_le['itemid'].value_counts())


df_le = pd.merge(df_le[['hadm_id','itemid','charttime','valuenum']],
         df_icustay, on='hadm_id',how='inner')

df_che = pd.merge(df_che[['hadm_id','itemid','charttime','valuenum']],
         df_icustay, on='hadm_id',how='inner')

In [ ]:
# filtering icu stays, reset charttime to start from zero (icu admission)

df_le['temp1'] = round((df_le['charttime'] - df_le['intime']) / np.timedelta64(1, 's')/3600,2)
df_le['temp2'] = round((df_le['outtime'] - df_le['charttime']) / np.timedelta64(1, 's')/3600,2)
df_le = df_le[ (df_le['temp1']>0) & (df_le['temp2']>0) ]
df_le['charttime'] = df_le['temp1']
df_le = df_le.sort_values(['stay_id','itemid', 'charttime'])

df_che['temp1'] = round((df_che['charttime'] - df_che['intime']) / np.timedelta64(1, 's')/3600,2)
df_che['temp2'] = round((df_che['outtime'] - df_che['charttime']) / np.timedelta64(1, 's')/3600,2)
df_che = df_che[ (df_che['temp1']>0) & (df_che['temp2']>0) ]
df_che['charttime'] = df_che['temp1']
df_che = df_che.sort_values(['stay_id','itemid', 'charttime'])




In [ ]:
#convert charttime(int) to hours, and resample and forward fill imputation

from datetime import datetime


df_le = df_le[['stay_id','itemid','charttime','valuenum']][:]
df1=df_le
df_le['charttime'] = datetime(2000,1,1,0,0,0) + df_le.charttime.astype('timedelta64[h]')
df_le = df_le.set_index('charttime')
# print(df_le.index)
df_le= df_le.groupby(['stay_id','itemid']).resample(grid)['valuenum'].last().reset_index(drop=False)
df_le['valuenum']= df_le.groupby(['stay_id','itemid'])['valuenum'].fillna(method='ffill')
df_le = df_le.sort_values(['stay_id','itemid', 'charttime'])



df_che = df_che[['stay_id','itemid','charttime','valuenum']][:]
df1=df_che
df_che['charttime'] = datetime(2000,1,1,0,0,0) + df_che.charttime.astype('timedelta64[h]')
df_che = df_che.set_index('charttime')
# print(df_che.index)
df_che= df_che.groupby(['stay_id','itemid']).resample(grid)['valuenum'].last().reset_index(drop=False)
df_che['valuenum']= df_che.groupby(['stay_id','itemid'])['valuenum'].fillna(method='ffill')
df_che = df_che.sort_values(['stay_id','itemid', 'charttime'])


In [ ]:
# pivot dataframes so that each measurement is a column
# concatenate df_le and df_che

df_lee = df_le.set_index(['charttime','stay_id'])
df_lee = df_lee.pivot(columns = 'itemid', values='valuenum')

df_chee = df_che.set_index(['charttime','stay_id'])
df_chee = df_chee.pivot(columns = 'itemid', values='valuenum')

df = pd.concat([df_lee, df_chee], axis=1)
# df = df_chee


df = df.reset_index(drop=False)
df = df.sort_values(['stay_id', 'charttime'])


print(df.isnull().sum())
print(len(df))


print(df.dropna().isnull().sum())
print(len(df.dropna()))

df = df.dropna(subset=[51003])


In [ ]:
df
print(df['stay_id'].value_counts())
sample_id = df['stay_id'].value_counts().index[0]
print(sample_id)

In [ ]:
# plot before normalization

stay_ids = df_icustay['stay_id'].unique().tolist()
# print(df['stay_id'].value_counts())


df_temp = df[df['stay_id']==sample_id ]
fig = px.line(df_temp, x='charttime', y=[220045, 220210, 51003], markers=True)
fig.show()


# df_temp = df_le[df_le['stay_id']==39132344    ]
# fig = px.line(df_temp, x='charttime', y='valuenum', color='itemid', markers=True)
# fig.show()


# df_temp = df_che[df_che['stay_id']==39132344    ]
# fig = px.line(df_temp, x='charttime', y='valuenum', color='itemid', markers=True)
# fig.show()

In [ ]:
df_temp

In [ ]:
# normalizing the data

from sklearn.preprocessing import MinMaxScaler
# define min max scaler
scaler = MinMaxScaler()
# transform data
df_temp = df_temp.set_index(['charttime','stay_id'])

df_temp = pd.DataFrame(scaler.fit_transform(df_temp), index=df_temp.index, columns=df_temp.columns)
df_temp = df_temp.reset_index(drop=False)


In [ ]:
# plot after normalization

print(df_temp.columns)
fig = px.line(df_temp, x='charttime', y=[       220045,    220210,  ], markers=True)
fig.show()


In [ ]:
from sklearn.model_selection import GroupShuffleSplit
splitter = GroupShuffleSplit(test_size=.20, n_splits=1, random_state = 7)
split = splitter.split(df, groups=df['stay_id'])
train_inds, test_inds = next(split)

df_train = df.iloc[train_inds]
df_test = df.iloc[test_inds]

print(len(df['stay_id'].unique()))
print(len(df_train['stay_id'].unique()))
print(len(df_test['stay_id'].unique()))

In [ ]:
def split_series(series, n_past, n_future):
  #
  # n_past ==> no of past observations
  #
  # n_future ==> no of future observations 
  #
  X, y = list(), list()
  for window_start in range(len(series)):
    past_end = window_start + n_past
    future_end = past_end + n_future
    if future_end > len(series):
      break
    # slicing the past and future parts of the window
    past, future = series[window_start:past_end, :], series[past_end:future_end, :]
    X.append(past)
    y.append(future)
  return np.array(X), np.array(y)

n_past = 3  # * grid
n_future = 2 # * grid
n_features = 2




# convert train dataframe to numpy
stay_ids = df_train['stay_id'].unique()
list_X = []
list_y = []
for i in stay_ids:
  df_temp = df_train[df_train['stay_id']==i].drop(columns=['stay_id','charttime']).to_numpy()
  X_train, y_train = split_series(df_temp ,n_past, n_future)
  
  list_X.append(X_train)
  list_y.append(y_train)

list_X = [i for i in list_X if i.ndim==3]
list_y = [i for i in list_y if i.ndim==3]

X_train=np.concatenate(list_X, axis = 0)
y_train=np.concatenate(list_y, axis = 0)



# convert test dataframe to numpy
stay_ids = df_test['stay_id'].unique()
list_X = []
list_y = []
for i in stay_ids:
  df_temp = df_test[df_test['stay_id']==i].drop(columns=['stay_id','charttime']).to_numpy()
  X_test, y_test = split_series(df_temp ,n_past, n_future)

  list_X.append(X_test)
  list_y.append(y_test)

list_X = [i for i in list_X if i.ndim==3]
list_y = [i for i in list_y if i.ndim==3]

X_test=np.concatenate(list_X, axis = 0)
y_test=np.concatenate(list_y, axis = 0)


print(f""" 
X_train.shape = {X_train.shape}
y_train.shape = {y_train.shape}
X_test.shape = {X_test.shape}
y_test.shape = {y_test.shape}
 """)


In [ ]:

import tensorflow as tf
import os
n_features=X_train.shape[-1]
n_past=X_train.shape[-2]
n_future=y_train.shape[-1]

print(X_train.shape, X_test.shape)

In [ ]:
encoder = tf.keras.models.Sequential([
    # tf.keras.layers.LSTM(100, return_state=True, input_shape = (n_past,n_features)),
    tf.keras.layers.Flatten(input_shape=[3, 2]),
    tf.keras.layers.Dense(100, activation="relu"),
    tf.keras.layers.Dense(30, activation="relu"),
])

In [ ]:
# E1D1
# n_features ==> no of features at each timestep in the data.
#
encoder_inputs = tf.keras.layers.Input(shape=(n_past, n_features))
encoder_l1 = tf.keras.layers.LSTM(100, return_state=True)
encoder_outputs1 = encoder_l1(encoder_inputs)

encoder_states1 = encoder_outputs1[1:]

#
decoder_inputs = tf.keras.layers.RepeatVector(n_future)(encoder_outputs1[0])

#
decoder_l1 = tf.keras.layers.LSTM(100, return_sequences=True)(decoder_inputs,initial_state = encoder_states1)
decoder_outputs1 = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(n_features))(decoder_l1)

#
model_e1d1 = tf.keras.models.Model(encoder_inputs,decoder_outputs1)

#
model_e1d1.summary()

In [ ]:
reduce_lr = tf.keras.callbacks.LearningRateScheduler(lambda x: 1e-3 * 0.90 ** x)
model_e1d1.compile(optimizer=tf.keras.optimizers.Adam(), loss=tf.keras.losses.Huber())
history_e1d1=model_e1d1.fit(X_train,y_train,epochs=25,validation_data=(X_test,y_test),batch_size=32,verbose=1,callbacks=[reduce_lr])


In [ ]:
print(history_e1d1.history['loss'])
plt.plot(history_e1d1.history['loss'])
plt.plot(history_e1d1.history['val_loss'])
plt.title("E1D1 Model Loss")
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend(['Train', 'Valid'])
plt.show()

In [ ]:
pred1_e1d1=model_e1d1.predict(X_test)
# pred1_e2d2=model_e2d2.predict(X_test)

pred_e1d1=model_e1d1.predict(X_train)
# pred_e2d2=model_e2d2.predict(X_train)

In [ ]:
print(pred1_e1d1.shape)
print(y_test.shape)
temp_test = y_test.reshape(-1,2)
temp_pred = pred1_e1d1.reshape(-1,2)


print(temp_pred.shape)
df_temp = pd.DataFrame(temp_pred[:1000,:],columns=[       220045,    220210,  ])
fig = px.line(df_temp, y=[       220045,    220210,  ], markers=True)

df_temp = pd.DataFrame(temp_test[:1000,:],columns=[       220045,    220210,  ])
fig = px.line(df_temp, y=[       220045,    220210,  ], markers=True)


fig.show()

In [ ]:
normal_df = df
RANDOM_SEED = 1374

In [ ]:
from sklearn.model_selection import train_test_split

train_df, val_df = train_test_split(
  normal_df,
  test_size=0.15,
  random_state=RANDOM_SEED
)
val_df, test_df = train_test_split(
  val_df,
  test_size=0.33,
  random_state=RANDOM_SEED
)

In [ ]:
# Data preprocessing
def create_dataset(df):
  sequences = df.astype(np.float32).to_numpy().tolist()
  dataset = [torch.tensor(s).unsqueeze(1).float() for s in sequences]
  n_seq, seq_len, n_features = torch.stack(dataset).shape
  return dataset, seq_len, n_features


train_dataset, seq_len, n_features = create_dataset(train_df)
val_dataset, _, _ = create_dataset(val_df)
test_normal_dataset, _, _ = create_dataset(test_df)
test_anomaly_dataset, _, _ = create_dataset(anomaly_df)

In [ ]:
import torch
import torch.nn as nn

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class Encoder(nn.Module):
  def __init__(self, seq_len, n_features, embedding_dim=64):
    super(Encoder, self).__init__()
    self.seq_len, self.n_features = seq_len, n_features
    self.embedding_dim, self.hidden_dim = embedding_dim, 2 * embedding_dim
    self.rnn1 = nn.LSTM(
      input_size=n_features,
      hidden_size=self.hidden_dim,
      num_layers=1,
      batch_first=True
    )
    self.rnn2 = nn.LSTM(
      input_size=self.hidden_dim,
      hidden_size=embedding_dim,
      num_layers=1,
      batch_first=True
    )
  def forward(self, x):
    x = x.reshape((1, self.seq_len, self.n_features))
    x, (_, _) = self.rnn1(x)
    x, (hidden_n, _) = self.rnn2(x)
    return hidden_n.reshape((self.n_features, self.embedding_dim))


class Decoder(nn.Module):
  def __init__(self, seq_len, input_dim=64, n_features=1):
    super(Decoder, self).__init__()
    self.seq_len, self.input_dim = seq_len, input_dim
    self.hidden_dim, self.n_features = 2 * input_dim, n_features
    self.rnn1 = nn.LSTM(
      input_size=input_dim,
      hidden_size=input_dim,
      num_layers=1,
      batch_first=True
    )
    self.rnn2 = nn.LSTM(
      input_size=input_dim,
      hidden_size=self.hidden_dim,
      num_layers=1,
      batch_first=True
    )
    self.output_layer = nn.Linear(self.hidden_dim, n_features)
  def forward(self, x):
    x = x.repeat(self.seq_len, self.n_features)
    x = x.reshape((self.n_features, self.seq_len, self.input_dim))
    x, (hidden_n, cell_n) = self.rnn1(x)
    x, (hidden_n, cell_n) = self.rnn2(x)
    x = x.reshape((self.seq_len, self.hidden_dim))
    return self.output_layer(x)

class RecurrentAutoencoder(nn.Module):
  def __init__(self, seq_len, n_features, embedding_dim=64):
    super(RecurrentAutoencoder, self).__init__()
    self.encoder = Encoder(seq_len, n_features, embedding_dim).to(device)
    self.decoder = Decoder(seq_len, embedding_dim, n_features).to(device)
  def forward(self, x):
    x = self.encoder(x)
    x = self.decoder(x)
    return x



In [ ]:
import copy

def train_model(model, train_dataset, val_dataset, n_epochs):
  optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
  criterion = nn.L1Loss(reduction='sum').to(device)
  history = dict(train=[], val=[])
  best_model_wts = copy.deepcopy(model.state_dict())
  best_loss = 10000.0
  for epoch in range(1, n_epochs + 1):
    model = model.train()
    train_losses = []
    for seq_true in train_dataset:
      optimizer.zero_grad()
      seq_true = seq_true.to(device)
      seq_pred = model(seq_true)
      loss = criterion(seq_pred, seq_true)
      loss.backward()
      optimizer.step()
      train_losses.append(loss.item())
    val_losses = []
    model = model.eval()
    with torch.no_grad():
      for seq_true in val_dataset:
        seq_true = seq_true.to(device)
        seq_pred = model(seq_true)
        loss = criterion(seq_pred, seq_true)
        val_losses.append(loss.item())
    train_loss = np.mean(train_losses)
    val_loss = np.mean(val_losses)
    history['train'].append(train_loss)
    history['val'].append(val_loss)
    if val_loss < best_loss:
      best_loss = val_loss
      best_model_wts = copy.deepcopy(model.state_dict())
    print(f'Epoch {epoch}: train loss {train_loss} val loss {val_loss}')
  model.load_state_dict(best_model_wts)
  return model.eval(), history

In [ ]:
model = RecurrentAutoencoder(seq_len, n_features, 128)
model = model.to(device)

model, history = train_model(
  model,
  train_dataset,
  val_dataset,
  n_epochs=150
)